# Workflow Tutorial for a JPSS (Polar-Orbiting) VIIRS Level 2 (Granule) Data File

This tutorial was written in February 2023 by Dr. Amy Huff, IMSG at NOAA/NESDIS/STAR (amy.huff@noaa.gov). It demonstrates a practical example of how to use the **Xarray**, **netCDF4**, **NumPy**, **Matplotlib**, and **Cartopy** Python packages to work with a VIIRS Level 2 (granule) file in netCDF4 (.nc) format.  

The main workflow steps are:
- Open the VIIRS .nc file
- Read the global file metadata
    - Recognize when data variables are organized into groups
    - Find names of groups, open a group & read the metadata for variables in the group
- Visualize point satellite data on a map with an image tile background
    - Format data markers
    - Format map plot title automatically using information from satellite file name
- Save map image file

## Step 0: Read data product documentation

Before working with a new dataset, it is **imperative to find and read the documentation for the data**. Documentation contains important information on how to work correctly with a dataset, such as the valid data range, spatial coverage, spatial and temporal resolution, data quality flags, etc.

In this tutorial, we will be working with a **NOAA-20 VIIRS Active Fires (AF) I-band** data file. 
- NOAA-20 is a polar-orbiting satellite, part of NOAA's Joint Polar Satellite Series (JPSS)
- VIIRS is the Visible Infrared Imaging Radiometer Suite, a sensor on the NOAA-20, SNPP, and NOAA-21 satellites
- VIIRS AF I-band is a Level 2 product, called a VIIRS Environmental Data Record (EDR)
    - EDRs are derived from VIIRS Level 1b data (radiances), called VIIRS Sensor Data Records (SDRs)
    - EDRs are distributed as [granules](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/satellite_data_viirs_granules.php), an observation area that spans approximately 85 seconds and covers approximately 3040 km x 570 km
- VIIRS AF I-band data has 375 m spatial resolution with 2-4 observations per day for a given location at low & mid-latitudes (daytime and nighttime) (more observations at higher latitudes)

VIIRS AF I-band documentation:
- [User's Manual](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/documents/VIIRS_AF_Iband_Users_Manual_V1_20200615.pdf)
- [Algorithm Theoretical Basis Document (ATBD)](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/documents/VIIRS_AF_Iband_ATBD_v1.0_20200506.pdf)

## Step 1: Import Python packages

Five Python packages (libraries) and two Python modules are used in this Notebook:
- The **Xarray** library is used to work with labeled multi-dimensional arrays
- The **netCDF4** library is used to read and write netCDF4 files
- The **NumPy** library is used to perform array operations
- The **Matplotlib** library is used to make plots
- The **Cartopy** library is used to create maps
- The **datetime** module is used to manipulate dates and times
- The **pathlib** module is used to set filesystem paths for the user's operating system

In [ ]:
import xarray as xr

from netCDF4 import Dataset

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt

import cartopy.io.img_tiles as cimgt
from cartopy import crs as ccrs

import datetime

from pathlib import Path

## Step 2: Set the directory path for the satellite data file

It is good practice to set directory paths using the [pathlib module](https://docs.python.org/3/library/pathlib.html#module-pathlib), which automatically uses the correct format for the user's operating system. This helps avoid errors in situations when more than one person is using the same code file, because Windows uses back slashes in directory paths, while MacOS and Linux use forward slashes.

For this tutorial, the VIIRS data file is located in a directory on the training server.  The same **pathlib** syntax (```Path('directory_name')```) can be used to set the path for any directory, including a directory located on a local computer or a remote server.

In [ ]:
directory_name = '/tljh-data/sat_data/NOAA'
directory_path = Path(directory_name)

## Step 3: Open a VIIRS Active Fires (AF) I-band satellite data file using Xarray

Let's open the VIIRS Active Fires (AF) I-band satellite data file (```file_name```). The full path for the data file (```file_id```) is set using **pathlib** syntax.

To open a single satellite data file with **Xarray**, use ```xr.open_dataset()```. The contents of a data file are called a "Dataset" in **Xarray**, conventionally abbreviated as ```ds```. 

In [ ]:
file_name = 'AF-Iband_v1r0_j01_s202210162118082_e202210162119327_c202210162142235.nc'
file_id = directory_path / file_name

ds = xr.open_dataset(file_id, engine='netcdf4')

### Step 3.1: Print the file metadata using Xarray

Running the name of the Dataset (```ds```) will print the file metadata.

The global file metadata are listed under ```Attributes```. We can see that these VIIRS data come from the NOAA-20 satellite. The data in the file span the time period 21:18.08 to 21:19.32 UTC on 16 October 2022 and cover an area of approximately 36.5 to 46.8 °N latitude and 107.7 to 145.3 °W longitude.

Usually, **Xarray** displays the metadata for data contained in a netCDF4 file under ```Data variables```. But in this file, **Xarray** is displaying zero (0) ```Dimensions```, ```Coordinates```, ```Data variables```, and ```Indexes```.  If you encounter this situation with a netCDF4 file using **Xarray**, the file is probably organized into **groups**.

**Groups** are an [optional organization structure for netCDF4 files](https://docs.unidata.ucar.edu/netcdf-c/current/netcdf_data_model.html#enhanced_model), similar to directories in a Unix file system. Some NOAA satellite data files, such as the VIIRS Active Fires files, use **groups**, so you should know how to recognize them.

In [ ]:
ds

## Step 4: Open a VIIRS AF I-band satellite data file using netCDF4

**Xarray** can open **groups** in a netCDF4 file, but you need to know their names. Unfortunately, [Xarray can't read groups metadata](https://github.com/pydata/xarray/issues/4840), and thus can't display the names of the groups. This is one of the major shortcomings of using the **Xarray** package. 

As a work-around, the **groups** in a satellite file can be read using either the **netCDF4** Python package or NASA's free **Panoply** tool. 

**Panoply** is incredibly useful for making simple plots to check the contents of satellite data files, so if you are going to be working with satellite data on a regular basis, consider [installing Panoply](https://www.giss.nasa.gov/tools/panoply/download/) on your computer.

Let's look at the other option, using the ```Dataset()``` constructor of the **netCDF4** package to open the VIIRS AF I-band file. 

In [ ]:
root_group = Dataset(file_id)

### Step 4.1: Print the file metadata using netCDF4

The names of the **groups** are listed at the very end, underneath the global file metadata. Notice that the global file metadata are the same as we saw using **Xarray** in Step 3.1.

In [ ]:
root_group

### Step 4.2: Print the groups metadata using netCDF4

There are three groups in this file: "Fire Mask", "Fire Pixels", and "Metadata for OSPO Monitoring Tool". 

Let's print the "Fire Mask" and "Fire Pixels" group metadata to see what data variables are in the groups.

In [ ]:
root_group.groups['Fire Mask']

In [ ]:
root_group.groups['Fire Pixels']

### Step 4.3: Close the file opened using netCDF4

Now that we know the names of the groups in the file, and we have identified that the fire detections data are in the "Fire Pixels" group, it is good practice to close the satellite file we opened using **netCDF4**.

In [ ]:
root_group.close()

## Step 5: Open the "Fire Pixels" group using Xarray

Now we can go back to **Xarray** and open the "Fire Pixels" group using the ```group='Fire Pixels'``` argument in ```xr.open_dataset()```.  To avoid confusion, let's assign the "Fire Pixels" Dataset a name that's different ("ds_group_pixels") from the overall file Dataset ("ds").

In [ ]:
ds_group_pixels = xr.open_dataset(file_id, group='Fire Pixels', engine='netcdf4')

### Step 5.1: Print the metadata for the "Fire Pixels" group using Xarray

The satellite data in the file are displayed under ```Data variables```.  A data variable is called a "DataArray" in **xarray**, conventionally abbreviated as ```da```.

There are 25 ```Data variables``` in this VIIRS file. They are one-dimensional arrays with ```Dimensions``` of ```nfire```, the number of fires detected, which is 161 fires in this file.

```FP_latitude``` (fire pixel latitude) and ```FP_longitude``` (fire pixel longitude) are the variables we will focus on for this tutorial. Clicking on the data attributes icon for these two variables shows the "long name" for the arrays.

In [ ]:
ds_group_pixels

### Step 5.2: Open the FP_latitude and FP_longitude data arrays as NumPy arrays to see values

For some satellite files, the data arrays are too long to be displayed by the data repository icon in the ```Data variables``` metadata for the entire group. When working with a new data file, it's good practice to take a look at the data arrays before proceeding to analyze or visualize the data.

Let's use the ```xr.DataArray.values``` function to open the ```FP_latitude``` and ```FP_longitude``` data arrays as **NumPy** arrays, and print the arrays so we can see the exact latitude and longitude values for the fire pixels. 

In [ ]:
ds_group_pixels.FP_latitude.values

In [ ]:
ds_group_pixels.FP_longitude.values

## <span style='color:blue'>Exercise 1: Open the FP_power data array as a NumPy array</span>

In the cell below, print the values of the "FP_power" data array.

What is the "long name" of the "FP_power" data variable?  What are the units of the "FP_power" data?

In [ ]:
## Exercise 1: Open the "FP_power" data array as a NumPy array to see the values



<details><summary><b><font color="blue">Click here for the solution to Exercise 1</font></b></summary>
    <p></p>

<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_group_pixels<span style="color: #666666">.</span>FP_power<span style="color: #666666">.</span>values
</pre></div>

</details>

## <span style='color:blue'>Exercise 2: Open the "Fire Mask" group using Xarray & print the group metadata</span>

In the cells below, open the "Fire Mask" group using **Xarray** and print the group metadata. To avoid confusion, give the "Fire Mask" Dataset a unique name ("ds_group_mask").

What are the **dimensions** of the **Data variables** in the group?

In [ ]:
## Exercise 2: Open the "Fire Mask" group using Xarray

ds_group_mask = 

In [ ]:
## Exercise 2: Print the "Fire Mask" group metadata



<details><summary><b><font color="blue">Click here for the solution to Exercise 2</font></b></summary>
    <p></p>

<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_group_mask <span style="color: #666666">=</span> xr<span style="color: #666666">.</span>open_dataset(file_id, group<span style="color: #666666">=</span><span style="color: #BA2121">&#39;Fire Mask&#39;</span>, engine<span style="color: #666666">=</span><span style="color: #BA2121">&#39;netcdf4&#39;</span>)
</pre></div>
    
<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_group_mask
</pre></div>

</details>

## Step 6: Make a quick scatter plot of Fire Pixel latitude vs. longitude using Xarray

When working with a new data product, it's good practice to make a quick plot to check the distribution of data in the file. An easy way to do this is by using [Xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html), which are built on top of **Matplotlib**.

Since the VIIRS fire data are point data, let's make a [scatter plot](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.plot.scatter.html). The [tutorial for the Xarray scatter plotting function](https://docs.xarray.dev/en/stable/user-guide/plotting.html#scatter) has some useful examples as a reference.

The arguments for the [Matplotlib scatter plot function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) are used to plot the fire detections as red square markers.
- Marker colors can be selected from [Matplotlib's standard colors](https://matplotlib.org/stable/gallery/color/named_colors.html) 
- Marker style is formatted using [Matplotlib's marker settings](https://matplotlib.org/stable/api/markers_api.html)

For this tutorial, we will make a map that zooms in on the [Nakia Creek Fire](https://inciweb.nwcg.gov/incident-information/wapcs-nakia-creek-fire) and the [Siouxon and Sunset Fires](https://inciweb.nwcg.gov/incident-information/wagpf-siouxon-and-sunset-fires) in southern Washington state, near the border with Oregon. These fires are the cluster of data in the upper left side of the plot.

In [ ]:
ds_group_pixels.plot.scatter(x='FP_longitude', y='FP_latitude', c='r', marker='s')

## <span style='color:blue'>Exercise 3: Change the marker style & color for the quick scatter plot using Xarray</span>

In the cell below, make a quick scatter plot of fire pixel latitude vs. fire pixel longitude, with magenta-colored, diamond-shaped markers.

If you have extra time, try some different colors and/or different marker types.

In [ ]:
## Exercise 3: Make a quick scatter plot of fire pixel latitude vs. longitude (magenta diamond markers)



<details><summary><b><font color="blue">Click here for the solution to Exercise 3</font></b></summary>
    <p></p>

<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_group_pixels<span style="color: #666666">.</span>plot<span style="color: #666666">.</span>scatter(x<span style="color: #666666">=</span><span style="color: #BA2121">&#39;FP_longitude&#39;</span>, y<span style="color: #666666">=</span><span style="color: #BA2121">&#39;FP_latitude&#39;</span>, c<span style="color: #666666">=</span><span style="color: #BA2121">&#39;magenta&#39;</span>, marker<span style="color: #666666">=</span><span style="color: #BA2121">&#39;D&#39;</span>)
</pre></div>

</details>

## <span style='color:blue'>BONUS Exercise: Make a quick scatter plot of Fire Radiative Power using Xarray</span>

In the cell below, make a quick scatter plot showing the value of fire radiative power ("FP_power") for each location of fire pixel latitude and longitude.  

Hint: delete the ```c=``` argument and add the ```hue=``` and ```cmap=``` arguments for the [Xarray scatter plot function](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.plot.scatter.html) to shade the plot of "FP_latitude" vs. "FP_longitude" by the value of "FP_power" using a colormap of your choice (I like "autumn" reversed).
- Select a sequential colormap ("cmap") from [Matplotlib's standard colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) 

In [ ]:
## BONUS exercise: Make a quick scatter plot of fire radiative power



<details><summary><b><font color="blue">Click here for the solution to the BONUS Exercise</font></b></summary>
    <p></p>

<div style="background: #f8f8f8; overflow:auto;width:auto;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">ds_group_pixels<span style="color: #666666">.</span>plot<span style="color: #666666">.</span>scatter(x<span style="color: #666666">=</span><span style="color: #BA2121">&#39;FP_longitude&#39;</span>, y<span style="color: #666666">=</span><span style="color: #BA2121">&#39;FP_latitude&#39;</span>, hue<span style="color: #666666">=</span><span style="color: #BA2121">&#39;FP_power&#39;</span>, marker<span style="color: #666666">=</span><span style="color: #BA2121">&#39;s&#39;</span>, cmap<span style="color: #666666">=</span><span style="color: #BA2121">&#39;autumn_r&#39;</span>)
</pre></div>

</details>

## Step 7: Plot a map background image using Matplotlib & Cartopy

When plotting point data, like satellite fire detections, an image map background provides geographic context for the data. **Cartopy** has many built-in [image tile interfaces](https://scitools.org.uk/cartopy/docs/latest/reference/io.html#image-tiles) for adding static images and map backgrounds. The Cartopy documentation has a [short tutorial that demonstrates how to plot a map background image](https://scitools.org.uk/cartopy/docs/latest/gallery/scalar_data/eyja_volcano.html#sphx-glr-gallery-scalar-data-eyja-volcano-py), which is a useful reference.

A nice map background option for satellite data is the ESRI World Street Map, which shows both political and geographic map features. Let's plot the ESRI World Street Map background for the region near the Washington/Oregon border where the Nakia Creek Fire and the Siouxon and Sunset Fires were burning on 16 October 2022. In this step, we will plot only the image tiles, without the satellite fire data, to see the map background clearly.

The image tiles (```map_background```) are loaded using **Cartopy's** web tiles interface (```cimgt.GoogleTiles(url=esri_tiles)```); the ```url=esri_tiles``` argument points to the URL for the ESRI World Street Map tiles. 

We plot the image tiles using **Matplotlib** by defining a figure and then adding ```geoaxes``` and a map projection using **Cartopy**. For best results, set the **map projection** for the figure to be the same as that of the ESRI map background (which happens to be **Mercator**). 

The ```map_background``` is plotted on the geoaxes using ```ax.add_image```. The ```zoom_level``` variable sets the resolution of ESRI map tiles; the higher the zoom level, the greater the map detail.

The domain of the map (in this case, the region near Vancouver, Washington & Portland, Oregon) is set using ```ax.set_extent([map_corners])``` where the ```[map_corners]``` are the ```[western_longitude, eastern_longitude, southern_latitude, northern_latitude]``` of the map boundaries in degrees (use negative values to indicate °S latitude or °W longitude). Specifying the **Plate Carree** map projection using the argument ```crs=ccrs.PlateCarree()``` tells **Cartopy** that the values of ```[map_corners]``` are entered in geographic coordinates (i.e., latitude and longitude).

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

plt.show()

## <span style='color:blue'>Exercise 4: Change the zoom level of the map background image</span>

In the cell below, fill in the missing ```zoom_level=``` variable for the ESRI map tiles, and plot the map background image. Try a few different zoom levels to see how they affect the resolution of the map.

In [ ]:
## Exercise 4: Change the "zoom_level" of the ESRI map tiles

esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

# SET ZOOM LEVEL
zoom_level = 
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

plt.show()

## Step 8: Plot VIIRS fire detections on a map background image using Matplotlib & Cartopy

Now that we know how to plot a map background image, we can plot the satellite fire detections on top of the map image.

First, we use the code from Step 7 to plot the map background image. Then we plot the fire detections (fire pixel latitude and longitude) using [Matplotlib's simple ```pyplot.plot``` plotting function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html).
- **Xarray** makes it easy to plot Data Arrays using ```xarray.DataArray```, e.g. ```ds_group_pixels.FP_longitude```, ```ds_group_pixels.FP_latitude```
- The ```pyplot.plot``` [Line 2D keyword arguments](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D) are used to customize the appearance of the fire detection markers as small red filled squares with a thin black edge

The plotting function argument ```transform=ccrs.PlateCarree()``` tells **Cartopy** that the VIIRS fire data are in geographic coordinates (i.e., latitude and longitude). This argument **must** be included when plotting satellite data that are in geographic coordinates, or the data will not plot correctly on the map projection.

We also add a [plot title](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.title.html) that is automatically generated by extracting information from the data file name (```file_id.name```) using [```str.split()```](https://docs.python.org/3/library/stdtypes.html#str.split) and indexing. The [pathlib ```Pure_Path.name``` method](https://docs.python.org/3/library/pathlib.html#pathlib.PurePath.name) returns a string representing the final path component, which is used to obtain the VIIRS data file name as a string.

The plot title includes the observation date and time in UTC.  The 8-digit observation date from the data file name is converted from "YYYYMMDD" format into a more user-friendly format ("DD Mon YYYY") using the **datetime** module's [```strftime``` and ```strptime``` format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

ax.plot(ds_group_pixels.FP_longitude, ds_group_pixels.FP_latitude, c='r', marker='s', lw=0, mec='k', mew=0.2, ms=3, 
        transform=ccrs.PlateCarree())

observation_date = datetime.datetime.strptime(file_id.name.split('_')[3][1:9], '%Y%m%d').date()
observation_date = observation_date.strftime('%d %b %Y')
title = 'NOAA-20/VIIRS Active Fires I-band  ' + observation_date + ' ' + file_id.name.split('_')[3][9:11] + ':' + file_id.name.split('_')[3][11:13] + ' UTC'
plt.title(title, pad=5, size=8, weight='bold')

plt.show()

## <span style='color:blue'>Exercise 5: Change the marker style & color for the map of VIIRS fire detections</span>

In the code block below, fill in the missing ```c=```, ```marker=```, ```mec=```, ```mew=```, and ```ms=``` arguments in the ```ax.plot``` command, and generate the map of VIIRS fire detections. Try a few different marker/edge color combinations.

Note: Keep the ```lw=0``` or **Matplotlib** will draw a line to connect the fire markers.

The [Line 2D instance](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D) contains the full list of plot properties for **Matplotlib's** ```pyplot.plot``` function.
- Marker colors can be selected from [Matplotlib's standard colors](https://matplotlib.org/stable/gallery/color/named_colors.html) 
- Marker style is formatted using [Matplotlib's marker settings](https://matplotlib.org/stable/api/markers_api.html)

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

# SET FIRE MARKER PLOT ARGUMENTS: "c=", "marker=", "mec=", "mew=", "ms="
ax.plot(ds_group_pixels.FP_longitude, ds_group_pixels.FP_latitude, c=, marker=, lw=0, mec=, mew=, ms=, 
        transform=ccrs.PlateCarree())

observation_date = datetime.datetime.strptime(file_id.name.split('_')[3][1:9], '%Y%m%d').date()
observation_date = observation_date.strftime('%d %b %Y')
title = 'NOAA-20/VIIRS Active Fires I-band  ' + observation_date + ' ' + file_id.name.split('_')[3][9:11] + ':' + file_id.name.split('_')[3][11:13] + ' UTC'
plt.title(title, pad=5, size=8, weight='bold')

plt.show()

## Step 9: Save the plot of VIIRS fire detections on a map background image using Matplotlib

Now that we have created the map of satellite fire detections, we need to save the image file so we can use the image in a research paper, presentation, website, or social media.

Using the same approach as in Step 8 for the plot title, a name for the saved image file (```saved_file_name```) is automatically generated by extracting information from the satellite data file name, including the observation date and time.

The map can be saved as an image file using **Matplotlib's** [```fig.savefig``` function](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.savefig).
- Set the resolution of the saved image file using the ```dpi=``` argument. 
    - The higher the dpi, the higher the figure resolution, but the larger the file size and the longer it will take to save the file. 
    - For figures that include an image map background, set a very high dpi, such as ```dpi=1000```, so the details of the map background will be clear. 
- Set the background color for the plot using the ```facecolor``` argument.
- Set ```bbox_inches='tight'``` to minimize the bounding box around the figure (to zoom in "tight" on the plot).
- Specify the format of the saved file (```saved_file_format```). 
    - The default format is .png
    - File format options include .eps, .jpeg, .jpg, .pdf, .pgf, .png, .ps, .raw, .rgba, .svg, .svgz, .tif, .tiff

**Comments are included below in the full code to visualize VIIRS Level 2 fire detections, to notate each step in the process.**

In [ ]:
# Get ESRI World Street Map tiles using Cartopy's web tile retrieval
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

# Set up figure in Matplotlib
fig = plt.figure(figsize=(10, 10))

# Add axes to figure and set map projection to be same as ESRI World Street Map tiles (Mercator)
ax = plt.axes(projection=map_background.crs)

# Plot ESRI World Street Map as background at specified resolution
zoom_level = 11
ax.add_image(map_background, zoom_level)

# Set extent of map to zoom-in to area of interest
# Use negative values to indicate °W longitude, e.g., 100 °W = -100
# Use negative values to indicate °S latitude, e.g., 30 °S = -30
ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

# Plot fire detections using small filled red square markers with thin black border
# "transform=ccrs.PlateCarree()" argument is required b/c data are in geographic coordinates
ax.plot(ds_group_pixels.FP_longitude, ds_group_pixels.FP_latitude, c='red', marker='s', lw=0, mec='k', mew=0.2, ms=3, 
        transform=ccrs.PlateCarree())

# Create plot title automatically using information from file name
# Use datetime module to extract observation date & time and reformat
# "file_id.name" returns the VIIRS data file name as a string
observation_date = datetime.datetime.strptime(file_id.name.split('_')[3][1:9], '%Y%m%d').date()
observation_date = observation_date.strftime('%d %b %Y')
# Put extracted/reformated strings together to make image title
title = 'NOAA-20/VIIRS Active Fires I-band  ' + observation_date + ' ' + file_id.name.split('_')[3][9:11] + ':' + file_id.name.split('_')[3][11:13] + ' UTC'
# Add plot title
plt.title(title, pad=5, size=8, weight='bold')

# Show plot
plt.show()

# Save figure
# "dpi" is image resolution in dots per inch; use a high dpi (e.g., 1000) for figures w/map image background
# "bbox_inches=tight" sets a "tight" bounding box around saved image
saved_file_format = '.png'
saved_file_name = 'viirs_af_i-band_' + file_id.name.split('_')[3][1:9] + '_' + file_id.name.split('_')[3][9:13] + saved_file_format
fig.savefig(saved_file_name, facecolor='w', dpi=1000, bbox_inches='tight')

# Close plot
plt.close()